In [3]:
!pip install pypdf
!pip install chromadb
!pip install langchain
!pip install sentence-transformers
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to

In [4]:
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install -U llama-cpp-python --no-cache-dir

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm_ll2 = Ollama(base_url="http://localhost:11434",
                 model="llama2",
                 callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

In [6]:
model_path = '/Users/jamakase/Downloads/ggml-model-q4_K.gguf'
# from llama_cpp import Llama
# llm = Llama(model_path=model_path)
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.


In [7]:


from llama_cpp import Llama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = Llama(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    # callback_manager=callback_manager,
    verbose=True,
)

llama_model_loader: loaded meta data with 16 key-value pairs and 363 tensors from /Users/jamakase/Downloads/ggml-model-q4_K.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q6_K     [ 13824,  5

In [8]:
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader, CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

embedder_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

embeddings = HuggingFaceEmbeddings(model_name=embedder_name)

loader = TextLoader(file_path='test.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)

SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
SYSTEM_TOKEN = 1788
USER_TOKEN = 1404
BOT_TOKEN = 9225
LINEBREAK_TOKEN = 13

ROLE_TOKENS = {
    "user": USER_TOKEN,
    "bot": BOT_TOKEN,
    "system": SYSTEM_TOKEN
}


def get_message_tokens(model, role, content):
    message_tokens = model.tokenize(content.encode("utf-8"))
    message_tokens.insert(1, ROLE_TOKENS[role])
    message_tokens.insert(2, LINEBREAK_TOKEN)
    message_tokens.append(model.token_eos())
    return message_tokens


def get_system_tokens(model):
    system_message = {"role": "system", "content": SYSTEM_PROMPT}
    return get_message_tokens(model, **system_message)


def process_text(text):
    lines = text.split("\n")
    lines = [line for line in lines if len(line.strip()) > 2]
    text = "\n".join(lines).strip()
    if len(text) < 10:
        return None
    return text


splitted_docs = text_splitter.split_documents(documents)
fixed_documents = []
for doc in splitted_docs:
    doc.page_content = process_text(doc.page_content)
    if not doc.page_content:
        continue
    fixed_documents.append(doc)
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)

docsearch = Chroma.from_documents(fixed_documents, embeddings)
# retriever = docsearch.as_retriever()

In [ ]:
def retrieve(history, db, retrieved_docs, k_documents):
    if db:
        msg = history[-1][0]
        retriever = db.as_retriever(search_kwargs={"k": k_documents})
        docs = retriever.get_relevant_documents(msg)
        retrieved_docs = "\n\n".join([doc.page_content for doc in docs])
    return retrieved_docs

In [11]:
query = '''Как мне быть, если при установки штурвала контроллера на первую позицию тепловоз не едет'''

# retriever.get_relevant_documents(query)

# qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
# qa.run(query)

model = llm

max_new_tokens = 1500
chat_history = [[query]]
retireved_docs = retrieve(chat_history, docsearch, [], 5)
print(retireved_docs)

last_user_message=chat_history[-1][0]
if retireved_docs:
    last_user_message = f"Контекст: {retireved_docs}\n\nИспользуя контекст, ответь на вопрос: {last_user_message}"
tokens = get_system_tokens(model)[:]
tokens.append(LINEBREAK_TOKEN)
role_tokens = [model.token_bos(), BOT_TOKEN, LINEBREAK_TOKEN]
tokens.extend(role_tokens)

generator = model.generate(
    tokens,
    top_k=30,
    top_p=0.9,
    temp=0.1
)

message_tokens = get_message_tokens(model=model, role="user", content=last_user_message)
tokens.extend(message_tokens)

partial_text = ""
for i, token in enumerate(generator):
    if token == model.token_eos() or (max_new_tokens is not None and i >= max_new_tokens):
        break
    partial_text += model.detokenize([token]).decode("utf-8", "ignore")
    # chat_history[-1][1] = partial_text
    # yield history

partial_text

' bot\nЕсли при установке штурвала контролера на первую позицию тепловоз с места не трогается, то необходимо провести диагностику силовой цепи и регулятора дизеля. Если вы обнаружите повреждение в силовой цепи или нарушение привода объединенного регулятора дизеля, следует следовать инструкции по устранению неисправности. Если же проблема связана с реле заземления, то необходимо провести дополнительную диагностику и устранить причину возникновения сигнала о повреждении.'

Запрос пользователя: При установке штурвала контролера на первую позицию тепловоз с места не трогается; Возможная неисправность:Снимается нагрузка, но дополнительно включается сигнальная лампа "Реле заземления".; Вероятная причина: Сработало реле заземления. В силовой цепи - пробой на корпус или частичное разрушение изоляции.; Метод устранения: Осмотрите всю силовую цепь. Если повреждение не

Запрос пользователя: При установке штурвала контролера на первую позицию тепловоз с места не трогается; Возможная неисправность:Снимается нагрузка, но дополнительно включается сигнальная лампа "Реле заземления".; Вероятная причина: Сработало реле заземления. В силовой цепи - пробой на корпус или частичное разрушение изоляции.; Метод устранения: Осмотрите всю силовую цепь. Если повреждение не

Запрос пользователя: При установке штурвала контролера на первую позицию тепловоз с места не трогается; Возможная неисправность:Тепловоз приходит в движение, но по кА и кВ наблюдается обратная полярность.; Ве

Llama.generate: prefix-match hit


' bot\nЕсли при установке штурвала контролера на первую позицию тепловоз с места не трогается, то возможно несколько причин. Сначала стоит проверить силовую цепь и убедиться в отсутствии повреждений. Если это не помогло, то следует обратить внимание на реле заземления. Возможно, оно сработало из-за пробоя на корпус или частичного разрушения изоляции. В этом случае необходимо провести ремонт силовой цепи и заменить поврежденные элементы. Если проблема не в силовой цепи, то следует обратиться к специалистам для диагностики и устранения неисправностей.'